In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import pennylane as qml

In [ ]:
#CONTROL PARAMETERS CELL

name = ["Lagos","Valencia"]

split_size = 0.8
batch_size =  32

hidden_layers =  4
n_layers = hidden_layers + 3
input_size = 2**(hidden_layers+1)
dropout = 0.2
learning_rate = 1e-3

epochs = 50

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

data = np.load('Dati/dataset_{}_x_{}.npy'.format(name[0],name[1]))
label = np.load('Dati/labels_{}_x_{}.npy'.format(name[0],name[1]))
data = torch.tensor(data, dtype=torch.float32)
label = torch.tensor(label, dtype=torch.float32)
label = torch.reshape(label, [label.shape[0], 1])


dataset = TensorDataset(data,label)

train_size = int(split_size * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False)

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface='torch')
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [ ]:
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [ ]:
class BlockLayer(torch.nn.Module):
    def __init__(self,input_feat, output_feat, d_out = 0):
        super(BlockLayer,self).__init__()
        self.block = torch.nn.Sequential(
            torch.nn.Linear( input_feat, output_feat),
            torch.nn.ReLU(),
            torch.nn.Dropout( d_out )
        )
    def forward(self, x):
        x = self.block(x)
        return x


class HybridNetwork(torch.nn.Module):
    def __init__(self, hidden_layers, input_feat, d_out):
        super(HybridNetwork, self).__init__()
        self.layers = torch.nn.ModuleList()

        output_feat = int(input_feat/2)
        self.layers.append( torch.nn.Linear( len(data[0])  , input_feat) )
        self.layers.append(torch.nn.Tanh())
        self.layers.append(torch.nn.Dropout(0.3))

        for i in range (hidden_layers):
            self.layers.append( BlockLayer(input_feat, output_feat, d_out) )
            if output_feat == 1: break
            input_feat = output_feat
            output_feat = int(input_feat*0.8)

        self.layers.append( torch.nn.Linear(input_feat , 2) )
        self.layers.append(qlayer)
        self.layers.append(torch.nn.Linear(2,1))
        self.layers.append(torch.nn.Sigmoid())
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
      

In [ ]:
hybrid_model = HybridNetwork( hidden_layers , input_size, dropout )
opt = torch.optim.Adam(hybrid_model.parameters(), learning_rate)
lossFunction = torch.nn.BCEWithLogitsLoss()

In [ ]:
from sklearn.metrics import accuracy_score
def evaluate_model( model , loader , loss_fn  ):
    
    all_outputs = []
    all_labels = []

    # Disabilitare il calcolo dei gradienti per la valutazione
    with torch.no_grad():
        for data, label in loader:
            output = model(data)
            label = torch.reshape(label, [label.shape[0], 1])
            all_outputs.append(output)
            all_labels.append(label)

        # Convertire gli output in tensori per applicare softmax
        all_outputs_tensor = torch.cat(all_outputs)
        all_labels_tensor = torch.cat(all_labels)

        # Calcolare la probabilità con Softmax per classificazione
        output_probs = torch.sigmoid(all_outputs_tensor)
        predicted = torch.round(output_probs)
    
        # Calcolare l'accuratezza
        accuracy = accuracy_score(all_labels_tensor.numpy(), predicted.numpy())

        # Calcolare la perdita totale
        #all_outputs_tensor.requires_grad = True  # Re-enable gradients for loss calculation
        loss = lossFunction(all_outputs_tensor, all_labels_tensor).item()

        return accuracy , loss

    

In [ ]:
# This is the actual model training (and validation). It may take a while, depending on:
# epoch number, network architecture, dataset size.

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
# This is the actual model training (and validation). It may take a while, depending on:
# epoch number, network architecture, dataset size.

train_losses = np.zeros(epochs)
val_losses = np.zeros(epochs)

train_accuracies = np.zeros(epochs)
val_accuracies = np.zeros(epochs)
# training the model
for i in range(epochs):
    training_total_loss = []
    hybrid_model.train()  # Set the model to training mode
    for id_batch, (data, label) in enumerate(train_loader):
   
        opt.zero_grad(set_to_none=True)
        output = hybrid_model(data)
        label = torch.reshape(label, [label.shape[0], 1])
        loss = lossFunction(output, label)
        loss.backward()
        opt.step()
        training_total_loss.append(loss.item())

    hybrid_model.eval()
    train_accuracies[i] , train_losses[i] = evaluate_model( hybrid_model , train_loader, lossFunction)
    val_accuracies[i] , val_losses[i] = evaluate_model( hybrid_model , val_loader, lossFunction)
      
    #scheduler.step()

    print(
        "Epoch: {}\tTraining Loss: {:.4f}\tVal Loss: {:.4f}\tTraining Accuracy: {:.2f}%\tValidation Accuracy: {:.2f}%".format(
            i + 1, train_losses[i], val_losses[i], 100 * train_accuracies[i], 100 * val_accuracies[i]
        )
    )

In [ ]:
plt.plot(train_losses)
plt.plot(val_losses)
plt.legend(["Loss on train", "Loss on validation"])
plt.xlabel("Epoch")
plt.ylabel("Loss")

In [ ]:
plt.plot(train_accuracies)
plt.plot(val_accuracies)
plt.legend(["Accuracies on train", "Accuracies on validation"])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")